## Imports and Setup

In [1]:
import os
import pandas as pd
import geopandas as gpd
import logging
import joblib
import torch

import sys
sys.path.insert(0, "../src")
import sat_download

sys.path.insert(0, "../utils/")
import data_utils
import config_utils
import pred_utils
import embed_utils

%load_ext autoreload
%autoreload 2

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


## Load Config Files

In [2]:
iso_code = "SEN"
adm_level = "ADM2"
shapename = "Foundiougne"
spacing = 150
buffer_size = 150

cwd = os.path.dirname(os.getcwd())
data_config_file = os.path.join(cwd, "configs/data_config.yaml")
data_config = config_utils.load_config(data_config_file)

sat_config_file = os.path.join(cwd, "configs/sat_configs/sat_config_500x500_60cm.yaml")
sat_creds_file = os.path.join(cwd, "configs/sat_configs/issa_sat_creds.yaml")

sat_config = config_utils.load_config(sat_config_file)
sat_creds = config_utils.create_config(sat_creds_file)

## Generate Prediction Tiles

In [3]:
tiles = pred_utils.generate_pred_tiles(
    data_config, iso_code, spacing, buffer_size, adm_level, shapename
).reset_index()
tiles["UID"] = list(tiles.index)

out_dir = data_utils._makedir(os.path.join(cwd, "output", iso_code, "tiles"))
out_file = os.path.join(out_dir, f"{iso_code}_{shapename}.gpkg")
tiles[["geometry"]].to_file(out_file, driver="GPKG")

print(f"Total tiles: {tiles.shape}")
tiles.head(3)

100%|████████████████████| 36/36 [04:28<00:00,  7.45s/it]                                                               


Total tiles: (6977, 12)


,index,geometry,shapeName,shapeISO,shapeID,shapeGroup,shapeType,points,index_right,type,properties,UID
0,191015,"POLYGON ((-1800166.608 1538399.613, -1800166.6...",Foundiougne,,32543966B99473740404621,SEN,ADM2,POINT (-1800316.608 1538249.613),50224,Feature,"{'height': -1.0, 'confidence': -1.0}",0
1,191016,"POLYGON ((-1800166.608 1538549.613, -1800166.6...",Foundiougne,,32543966B99473740404621,SEN,ADM2,POINT (-1800316.608 1538399.613),50224,Feature,"{'height': -1.0, 'confidence': -1.0}",1
2,191457,"POLYGON ((-1800016.608 1538399.613, -1800016.6...",Foundiougne,,32543966B99473740404621,SEN,ADM2,POINT (-1800166.608 1538249.613),50224,Feature,"{'height': -1.0, 'confidence': -1.0}",2


## Download Satellite Images

In [4]:
data = tiles.copy()
data["geometry"] = data["points"]
sat_dir = os.path.join(cwd, "output", iso_code, "images", shapename)
sat_download.download_sat_images(sat_creds, sat_config, data=data, out_dir=sat_dir)

INFO:root:Data dimensions: (6977, 14), CRS: EPSG:3857
100%|████████████████████| 6977/6977 [2:02:17<00:00,  1.05s/it]                                                         


## CNN Model

In [ ]:
model_config_file = os.path.join(cwd, f"configs/cnn_configs/vgg16.yaml")
model_config = config_utils.load_config(model_config_file)

result = pred_utils.cnn_predict(tiles, iso_code, shapename, model_config, sat_dir, n_classes=2)

## ViT Model

In [7]:
temp = pd.read_csv("/home/itingzon.unicef/giga/output/SEN/embeddings/SEN_Foundiougne_dinov2_vitl14_embeds.csv")
temp.head(3)

,UID,0,1,2,3,4,5,6,7,8,...,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
0,0,-0.648580,0.812641,1.782041,0.807176,-3.058083,-1.385760,3.914548,-0.628538,0.135209,...,-0.008649,-1.208241,-0.556140,0.089280,0.474727,1.490318,-2.104061,2.907655,-1.761186,-0.607599
1,1,-0.699730,-0.506200,1.284234,0.239414,-3.430830,-2.106042,3.507945,-0.659336,0.407805,...,-0.126094,-0.827913,-0.731924,-1.200732,-0.637605,1.696903,-1.839007,2.544005,-2.341905,0.052718
2,2,-1.781758,0.943659,1.433975,0.744863,-2.330132,-0.812688,3.586650,-1.658279,-0.423982,...,0.714072,-1.961411,-0.747469,0.978079,1.232219,2.570768,-1.366836,2.529151,-1.476436,-0.096525


In [13]:
model_config_file = os.path.join(cwd, f"configs/model_configs/dinov2_vitl14-LR.yaml")
model_config = config_utils.load_config(model_config_file)
result = pred_utils.vit_pred(tiles, model_config, iso_code, shapename, sat_dir)

Using cache found in /home/itingzon.unicef/.cache/torch/hub/facebookresearch_dinov2_main
INFO:dinov2:using MLP layer as FFN
INFO:root:Device: cpu
INFO:root:Generating embeddings...
INFO:root:Reading file /home/itingzon.unicef/giga/output/SEN/embeddings/SEN_Foundiougne_dinov2_vitl14_embeds.csv
INFO:root:Loaded /home/itingzon.unicef/giga/exp/SEN-dinov2_vitl14-LR/SEN-dinov2_vitl14-LR.pkl
INFO:sklearnex: sklearn.utils.validation._assert_all_finite: running accelerated version on CPU
INFO:sklearnex:sklearn.utils.validation._assert_all_finite: running accelerated version on CPU
INFO:root:Generated /home/itingzon.unicef/giga/output/SEN/results/SEN_Foundiougne_dinov2_vitl14-LR_results.gpkg
